In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

root = '/content/drive/MyDrive/instacart-market-basket-analysis/'
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_pickle(root + 'Finaldata.pkl')
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,...,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,196,3,3,1.0,1.0,23.0,7.0,1.0,1.0,...,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
1,1,10258,3,3,1.0,2.666667,23.0,7.0,1.0,1.0,...,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
2,1,10326,1,0,0.0,5.0,28.0,5.0,0.0,0.0,...,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
3,1,12427,3,3,1.0,2.333333,23.0,7.0,1.0,1.0,...,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0
4,1,13032,1,1,1.0,5.0,20.0,7.0,1.0,0.0,...,13,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0


In [ ]:
def reduce_memory(df):

    """
    This function reduce the dataframe memory usage by converting it's type for easier handling.

    Parameters: Dataframe
    Return: Dataframe
    """

    start_mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")

    for col in df.columns:
        if df[col].dtypes in ["int64", "int32", "int16"]:

            cmin = df[col].min()
            cmax = df[col].max()

            if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)

            elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)

            elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)

        if df[col].dtypes in ["float64", "float32"]:

            cmin = df[col].min()
            cmax = df[col].max()

            if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)

            elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)

    print("")
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")

    return df

In [ ]:
df = reduce_memory(df)


Memory usage of properties dataframe is : 2480.697868347168  MB


<ipython-input-3-451e58444b8f>:33: RuntimeWarning: overflow encountered in cast
  if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
<ipython-input-3-451e58444b8f>:33: RuntimeWarning: overflow encountered in cast
  if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
<ipython-input-3-451e58444b8f>:33: RuntimeWarning: overflow encountered in cast
  if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:



___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  1652.154685974121  MB
This is  66.60039930920382 % of the initial size


In [ ]:
df['order_diff'] = df.order_number - df.last_ordered_in
df.drop(['user_id', 'product_id'], axis = 1, inplace = True)

In [ ]:
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1,order_number,...,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1,order_diff
0,3,3,1.0,1.0,23.0,7.0,1.0,1.0,1.0,11.0,...,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0,4.0
1,3,3,1.0,2.666667,23.0,7.0,1.0,1.0,1.0,11.0,...,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0,4.0
2,1,0,0.0,5.0,28.0,5.0,0.0,0.0,0.0,11.0,...,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0,6.0
3,3,3,1.0,2.333333,23.0,7.0,1.0,1.0,1.0,11.0,...,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0,4.0
4,1,1,1.0,5.0,20.0,7.0,1.0,0.0,0.0,11.0,...,0.722222,6.0,0.741667,5.0,8.0,5.0,0.6,0.625,1.0,4.0


In [ ]:
df.shape

(5171372, 62)

In [ ]:
label = 'reordered'
x_cols = df.columns.drop('reordered')


In [ ]:
X = df[x_cols]
y = df[label]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.25)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3878529, 61) (3878529,)
(1292843, 61) (1292843,)


In [ ]:
y_train.value_counts()

,count
reordered,
0.0,3256911
1.0,621618


In [ ]:
np.ceil(y_train.value_counts()[0]/y_train.value_counts()[1])


np.float64(6.0)

In [ ]:
y_test.value_counts()


,count
reordered,
0.0,1085637
1.0,207206


In [ ]:
# freeing memory
del df, X, y
gc.collect()


0

Neural Network model


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import History
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sc = MinMaxScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)


ValueError: Input X contains infinity or a value too large for dtype('float64').

In [ ]:
input_dim =  X_train_sc.shape[1]
input_dim

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
history = History()

classifier = Sequential()

classifier.add(Dense(units = 64, activation = 'relu', input_dim = input_dim))
classifier.add(Dense(units = 15, activation = 'relu'))
classifier.add(Dense(units = 4, activation = 'relu'))
classifier.add(Dense(units = 1, activation ='sigmoid'))

classifier.compile(optimizer = "adam", loss = 'binary_crossentropy', metrics = ['accuracy', f1_m, precision_m, recall_m])

classifier.summary()


In [ ]:
%%time
# fit the model
classifier.fit(X_train_sc, y_train, epochs=20, batch_size=512, validation_split=0.15, verbose=1,class_weight= {0:1, 1:10},
          callbacks = [history, keras.callbacks.EarlyStopping(monitor='val_loss',
                                                              min_delta=0, patience=10, verbose=0, mode='auto')])

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 4))

# Accuracy
ax[0].plot(history.history['acc'])
ax[0].plot(history.history['val_acc'])
ax[0].set_ylabel('Accuracy')
ax[0].set_xlabel('# Epoch')
ax[0].legend(['train', 'test'], loc='upper left')
ax[0].set_title('Accuracy')

# Loss
ax[1].plot(history.history['loss'])
ax[1].plot(history.history['val_loss'])
ax[1].set_ylabel('Loss')
ax[1].set_xlabel('# Epoch')
ax[1].legend(['train', 'test'], loc='upper left')
ax[1].set_title('Loss')

In [ ]:
probabilities = classifier.predict_proba(X_test_sc)
predictions = classifier.predict_classes(X_test_sc)

print ("\n Classification report : \n",classification_report(y_test, predictions))
print ("Accuracy   Score : ",accuracy_score(y_test, predictions))

#confusion matrix
conf_matrix = confusion_matrix(y_test,predictions)
plt.figure(figsize=(12,12))
plt.subplot(221)
sns.heatmap(conf_matrix, fmt = "d",annot=True, cmap='Blues')
b, t = plt.ylim()
plt.ylim(b + 0.5, t - 0.5)
plt.title('Confuion Matrix')
plt.ylabel('True Values')
plt.xlabel('Predicted Values')

#f1-score
f1 = f1_score(y_test, predictions)
print("F1 Score: ", f1)

#roc_auc_score
model_roc_auc = roc_auc_score(y_test,probabilities)
print ("Area under curve : ",model_roc_auc,"\n")
fpr,tpr,thresholds = roc_curve(y_test,probabilities)
gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
threshold = np.round(thresholds[ix],3)

plt.subplot(222)
plt.plot(fpr, tpr, color='darkorange', lw=1, label = "Auc : %.3f" %model_roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best Threshold:' + str(threshold))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")

plt.show()